In [ ]:
from tracing.rl.a3cmodel import A3CModel
from tracing.rl.actions import *
from tracing.selenium_utils.controls import Types

from pretrain_dataset import *

import tensorflow as tf
import tensorflow_hub as hub

import threading
import json
import random
from scipy import misc
import numpy as np
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from controls_pretrain import *

In [ ]:
controls = read_control_dataset()
encoder = cache_embeddings(controls)

In [ ]:
def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        xticklabels=labels,
        yticklabels=labels,
        vmin=0,
        vmax=1,
        cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")


def run_and_plot(messages):
    message_embeddings = encoder.encode(messages)
    plot_similarity(messages, message_embeddings, 90)

In [ ]:
messages = [ctrl['label'] for ctrl in controls[:10]]
run_and_plot(messages)

In [ ]:
tf.reset_default_graph()
session = tf.Session()

In [ ]:
a3c = A3CModel(len(Actions.actions), session = session, train_deep=True)
model = ControlPretrainModel(a3c, session, encoder)

session.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [ ]:
to_train_inputs = [ctrl for ctrl in controls if ctrl['possible_actions'][3] == 0 
                   and sum(ctrl['possible_actions']) == 1]
print(len(to_train_inputs))

In [ ]:
# Choose what to train
# 1. to_train - is full dataset
# 2. to_train_input - is small dataset for controls that aren't clickable
for epoch in range(40):
    model.train(to_train_inputs, lr=0.001, 
            l2 = 0.0001,
            epoch_start = epoch, 
            epoch_end = epoch + 1, batch_size = 20, dropout = 0.8)
    
    saver.save(session, './checkpoints/pretrain_checkpoint-{}'.format(epoch))